<a href="https://colab.research.google.com/github/LxYuan0420/nlp/blob/main/notebooks/Inference_with_NuExtract_large_for_Structural_Information_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
def predict_NuExtract(model, tokenizer, text, schema, example=["","",""]):
    schema = json.dumps(json.loads(schema), indent=4)
    input_llm =  "<|input|>\n### Template:\n" +  schema + "\n"
    for i in example:
      if i != "":
          input_llm += "### Example:\n"+ json.dumps(json.loads(i), indent=4)+"\n"

    input_llm +=  "### Text:\n"+text +"\n<|output|>\n"
    input_ids = tokenizer(input_llm, return_tensors="pt", truncation=True, max_length=4000).to("cuda")

    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)
    return output.split("<|output|>")[1].split("<|end-output|>")[0]

In [3]:
model = AutoModelForCausalLM.from_pretrained("numind/NuExtract", trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("numind/NuExtract", trust_remote_code=True)

model.to("cuda")

model.eval()


config.json:   0%|          | 0.00/986 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

#### Example 1 - Model Card

In [4]:
text = """We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: https://github.com/mistralai/mistral-src
Webpage: https://mistral.ai/news/announcing-mistral-7b/"""

schema = """{
    "Model": {
        "Name": "",
        "Number of parameters": "",
        "Number of token": "",
        "Architecture": []
    },
    "Usage": {
        "Use case": [],
        "Licence": ""
    }
}"""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)



{
    "Model": {
        "Name": "Mistral 7B",
        "Number of parameters": "7\u2013billion",
        "Number of token": "",
        "Architecture": [
            "grouped-query attention (GQA)",
            "sliding window attention (SWA)"
        ]
    },
    "Usage": {
        "Use case": [
            "reasoning",
            "mathematics",
            "code generation",
            "follow instructions"
        ],
        "Licence": "Apache 2.0"
    }
}



#### Example 2 - Historical Event

In [5]:
text = """We commemorate several pivotal moments in human history, showcasing the resilience, ingenuity, and progress of humanity.

First, the Apollo 11 mission stands as a monumental achievement. On July 20, 1969, astronauts Neil Armstrong and Buzz Aldrin became the first humans to set foot on the Moon, while Michael Collins orbited above. This mission, fulfilling President John F. Kennedy's goal, was broadcast live, capturing the awe of millions worldwide. The mission involved scientific experiments, including lunar sample collection and the deployment of instruments to study the Moon's environment. The Apollo program's legacy continues to inspire generations in the pursuit of space exploration.

Another significant event is the fall of the Berlin Wall on November 9, 1989. The Berlin Wall, erected in 1961, symbolized the Cold War's division of East and West Berlin. Its fall marked the beginning of the end for the Iron Curtain and led to the reunification of Germany. The event was celebrated globally, symbolizing the triumph of freedom and the end of decades-long political tension. The reunification process was complex, involving significant economic, social, and political changes, but it ultimately paved the way for a united Europe.

Additionally, the signing of the Treaty of Versailles on June 28, 1919, formally ended World War I. This treaty imposed heavy reparations and territorial losses on Germany, aiming to prevent future conflicts. However, it also sowed the seeds of resentment that eventually led to World War II. The Treaty of Versailles remains a critical study in the complexities of peace negotiations and the long-term impacts of punitive measures in international relations.

Further Reading:
- NASA Apollo Missions: https://www.nasa.gov/mission_pages/apollo/missions/index.html
- Smithsonian National Air and Space Museum: https://airandspace.si.edu/learn/highlighted-topics/apollo-program
- Fall of the Berlin Wall: https://www.history.com/topics/cold-war/berlin-wall
- Treaty of Versailles: https://www.history.com/topics/world-war-i/treaty-of-versailles-1
"""

schema = """{
    "Historical Events": [
        {
            "Event": {
                "Name": "",
                "Date": "",
                "Key Figures": [],
                "Significance": ""
            },
            "Details": {
                "Achievements": [],
                "Scientific Contributions": [],
                "Legacy": ""
            },
            "Resources": {
                "Links": []
            }
        }
    ]
}
"""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)


{
    "Historical Events": [
        {
            "Event": {
                "Name": "Apollo 11 mission",
                "Date": "July 20, 1969",
                "Key Figures": [
                    "Neil Armstrong",
                    "Buzz Aldrin",
                    "Michael Collins"
                ],
                "Significance": "fulfilling President John F. Kennedy's goal"
            },
            "Details": {
                "Achievements": [
                    "first humans to set foot on the Moon",
                    "lunar sample collection",
                    "deployment of instruments to study the Moon's environment"
                ],
                "Scientific Contributions": [],
                "Legacy": "inspire generations in the pursuit of space exploration"
            },
            "Resources": {
                "Links": [
                    "https://www.nasa.gov/mission_pages/apollo/missions/index.html",
                    "https://airandspace.si.

#### Example 3 - Resume Extraction

In [7]:
text = """Name: Jonathan Davis

Contact Information:
Email: jonathan.davis@example.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/jonathandavis
Address: 123 Main Street, Anytown, USA

Professional Summary:
Experienced Software Engineer with over 10 years of experience in developing and implementing software solutions. Proficient in multiple programming languages and frameworks, with a strong background in AI and machine learning. Demonstrated ability to lead teams and manage projects from conception to completion. Passionate about leveraging technology to solve complex problems and drive innovation.

Education:
- Master of Science in Computer Science, Stanford University, 2014
- Bachelor of Science in Computer Engineering, University of California, Berkeley, 2012

Professional Experience:

Senior Software Engineer | Tech Innovators Inc. | San Francisco, CA
June 2018 – Present
- Lead a team of engineers in developing scalable software solutions for clients in various industries.
- Implement AI and machine learning algorithms to enhance product features and improve user experience.
- Collaborate with cross-functional teams to design and deploy new applications and services.
- Conduct code reviews and provide mentorship to junior engineers.

Software Engineer | Creative Solutions LLC | San Francisco, CA
January 2015 – May 2018
- Developed and maintained web applications using Python, JavaScript, and Ruby on Rails.
- Designed and implemented database schemas for new projects.
- Worked closely with clients to gather requirements and ensure project success.
- Automated testing and deployment processes to improve efficiency.

Junior Software Engineer | NextGen Technologies | Palo Alto, CA
July 2012 – December 2014
- Assisted in the development of software applications and tools.
- Conducted debugging and troubleshooting to resolve software issues.
- Participated in team meetings and contributed to project planning and execution.
- Maintained and updated technical documentation.

Skills:
- Programming Languages: Python, Java, JavaScript, Ruby
- Frameworks: TensorFlow, PyTorch, React, Ruby on Rails
- Tools: Git, Docker, Jenkins, Kubernetes
- Soft Skills: Team Leadership, Project Management, Problem Solving, Communication

Certifications:
- Certified Kubernetes Administrator (CKA), 2020
- AWS Certified Solutions Architect, 2019

Projects:
- AI-Powered Customer Support Chatbot: Developed a chatbot using natural language processing to improve customer service efficiency.
- Predictive Maintenance System: Implemented machine learning models to predict equipment failures and schedule maintenance.

References:
Available upon request.
"""

schema = """{
    "Resume": {
        "Name": "",
        "Contact Information": {
            "Email": "",
            "Phone": ""
        },
        "Education": [
            {
                "Degree": "",
                "Institution": "",
                "Year": ""
            }
        ],
        "Working Experience": [
            {
                "Job Title": "",
                "Company": "",
                "Location": "",
                "Dates": ""
            }
        ],
        "Projects": [
            {
                "Project Name": "",
                "Description": ""
            }
        ]
    }
}
"""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)


{
    "Resume": {
        "Name": "Jonathan Davis",
        "Contact Information": {
            "Email": "jonathan.davis@example.com",
            "Phone": "(555) 123-4567"
        },
        "Education": [
            {
                "Degree": "Master of Science",
                "Institution": "Stanford University",
                "Year": "2014"
            },
            {
                "Degree": "Bachelor of Science",
                "Institution": "University of California, Berkeley",
                "Year": "2012"
            }
        ],
        "Working Experience": [
            {
                "Job Title": "Senior Software Engineer",
                "Company": "Tech Innovators Inc.",
                "Location": "San Francisco, CA",
                "Dates": "June 2018 \u2013 Present"
            },
            {
                "Job Title": "Software Engineer",
                "Company": "Creative Solutions LLC",
                "Location": "San Francisco, CA",
      

#### Example 4 - Historical Event but we frame it as a NER task

In [8]:
text = """We commemorate several pivotal moments in human history, showcasing the resilience, ingenuity, and progress of humanity.

First, the Apollo 11 mission stands as a monumental achievement. On July 20, 1969, astronauts Neil Armstrong and Buzz Aldrin became the first humans to set foot on the Moon, while Michael Collins orbited above. This mission, fulfilling President John F. Kennedy's goal, was broadcast live, capturing the awe of millions worldwide. The mission involved scientific experiments, including lunar sample collection and the deployment of instruments to study the Moon's environment. The Apollo program's legacy continues to inspire generations in the pursuit of space exploration.

Another significant event is the fall of the Berlin Wall on November 9, 1989. The Berlin Wall, erected in 1961, symbolized the Cold War's division of East and West Berlin. Its fall marked the beginning of the end for the Iron Curtain and led to the reunification of Germany. The event was celebrated globally, symbolizing the triumph of freedom and the end of decades-long political tension. The reunification process was complex, involving significant economic, social, and political changes, but it ultimately paved the way for a united Europe.

Additionally, the signing of the Treaty of Versailles on June 28, 1919, formally ended World War I. This treaty imposed heavy reparations and territorial losses on Germany, aiming to prevent future conflicts. However, it also sowed the seeds of resentment that eventually led to World War II. The Treaty of Versailles remains a critical study in the complexities of peace negotiations and the long-term impacts of punitive measures in international relations.

Further Reading:
- NASA Apollo Missions: https://www.nasa.gov/mission_pages/apollo/missions/index.html
- Smithsonian National Air and Space Museum: https://airandspace.si.edu/learn/highlighted-topics/apollo-program
- Fall of the Berlin Wall: https://www.history.com/topics/cold-war/berlin-wall
- Treaty of Versailles: https://www.history.com/topics/world-war-i/treaty-of-versailles-1
"""

schema = """{
    "Historical Events": {
        "Event Name": [],
        "Event Date": [],
        "Person Name": []
    }
}
"""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)


{
    "Historical Events": {
        "Event Name": [
            "Apollo 11 mission",
            "fall of the Berlin Wall",
            "signing of the Treaty of Versailles"
        ],
        "Event Date": [
            "July 20, 1969",
            "November 9, 1989",
            "June 28, 1919"
        ],
        "Person Name": [
            "Neil Armstrong",
            "Buzz Aldrin",
            "Michael Collins",
            "John F. Kennedy"
        ]
    }
}



#### Example 5 - Historial Event but we frame it as a NER task

In this example, we use traditional NER tags such as Person, Location, and Organization to determine if reframing this structural information extraction as a NER task would be successful. Additionally, we have introduced a new tag called 'Adjective' to increase the difficulty. This approach is inspired by GLiNER, a zero-shot NER model capable of capturing POS tags.

Comment:
- Interestingly, it returns 'Moon' as a location.
- Able to capture some adjectives, but not all of them.

In [10]:

text = """We commemorate several pivotal moments in human history, showcasing the resilience, ingenuity, and progress of humanity.

First, the Apollo 11 mission stands as a monumental achievement. On July 20, 1969, astronauts Neil Armstrong and Buzz Aldrin became the first humans to set foot on the Moon, while Michael Collins orbited above. This mission, fulfilling President John F. Kennedy's goal, was broadcast live, capturing the awe of millions worldwide. The mission involved scientific experiments, including lunar sample collection and the deployment of instruments to study the Moon's environment. The Apollo program's legacy continues to inspire generations in the pursuit of space exploration.

Another significant event is the fall of the Berlin Wall on November 9, 1989. The Berlin Wall, erected in 1961, symbolized the Cold War's division of East and West Berlin. Its fall marked the beginning of the end for the Iron Curtain and led to the reunification of Germany. The event was celebrated globally, symbolizing the triumph of freedom and the end of decades-long political tension. The reunification process was complex, involving significant economic, social, and political changes, but it ultimately paved the way for a united Europe.

Additionally, the signing of the Treaty of Versailles on June 28, 1919, formally ended World War I. This treaty imposed heavy reparations and territorial losses on Germany, aiming to prevent future conflicts. However, it also sowed the seeds of resentment that eventually led to World War II. The Treaty of Versailles remains a critical study in the complexities of peace negotiations and the long-term impacts of punitive measures in international relations.

Further Reading:
- NASA Apollo Missions: https://www.nasa.gov/mission_pages/apollo/missions/index.html
- Smithsonian National Air and Space Museum: https://airandspace.si.edu/learn/highlighted-topics/apollo-program
- Fall of the Berlin Wall: https://www.history.com/topics/cold-war/berlin-wall
- Treaty of Versailles: https://www.history.com/topics/world-war-i/treaty-of-versailles-1
"""

schema = """{
    "Person": [],
    "Date": [],
    "Organization": [],
    "Location": [],
    "Adjectives": []
}
"""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)


{
    "Person": [
        "Neil Armstrong",
        "Buzz Aldrin",
        "Michael Collins",
        "John F. Kennedy"
    ],
    "Date": [
        "July 20, 1969",
        "November 9, 1989",
        "June 28, 1919"
    ],
    "Organization": [
        "NASA",
        "Smithsonian National Air and Space Museum",
        "Treaty of Versailles"
    ],
    "Location": [
        "Moon",
        "Berlin",
        "Germany"
    ],
    "Adjectives": [
        "pivotal",
        "monumental",
        "significant",
        "triumph",
        "complex",
        "punitive"
    ]
}

